In [1]:
%load_ext autoreload
%autoreload 2

# Making systems

In [2]:
import molsysmt as msm

In [3]:
from molsysmt import systems
import numpy as np

In [4]:
molsys = msm.convert(systems['TcTIM']['1tcd.h5msm'], to_form='molsysmt.MolSys')

In [5]:
bonded_atoms = msm.get(molsys, element='atom', selection=[0, 1, 2, 3, 4, 5], bonded_atoms=True)